In [1]:
import xarray as xr
import numpy as np
from sklearn.linear_model import LinearRegression

from utils import detrend, monthly_anomalies

In [2]:
t_grid_file = xr.open_zarr("data/ORCA025.L46-KFS006_TroPA.zarr/")

## Data preparation
### Linear model output

In [3]:
eta = xr.open_zarr("data/lmmm_eta_nemo.zarr/").eta.sel(
    mode=slice(1, 3), lat=slice(-5,5),lon=slice(140,260)
)

### Nemo SSH 

In [4]:
ssh_fitting = t_grid_file.sossheig.sel(lat=0.,lon=slice(140.,260.))
ssh = t_grid_file.sossheig.interp(lat=eta.lat,lon=eta.lon)

In [5]:
ssha_fitting = detrend(monthly_anomalies(ssh_fitting).compute())

In [6]:
ssh_mon_anom = detrend(monthly_anomalies(ssh).compute())

### SSH from linear model

In [7]:
eta_mon = eta.assign_coords(
    {'time':(eta.time-np.timedelta64(1314900,'s')).data}
).interp(time=ssh_mon_anom.time).fillna(0.).compute()

In [8]:
eta_fitting = eta_mon.interp(lat=0)

In [9]:
def fitting(ssha, eta_x):
    mask = ~(
        np.isnan(ssha).all('time')
        | np.isnan(eta_x.sel(mode=1)).all('time')
    )
    a = np.vstack(
        [
            eta_x.sel(mode=k)[:, mask].values.flatten()
            for k in (1,2,3)
        ]
    ).T
    a_inv = np.linalg.pinv(a)

    b = ssha[:, mask].values.flatten()

    coeff = a_inv.dot(b)
    coeff = xr.DataArray(
        coeff,
        coords=(eta_x.sel(mode=slice(1, 3)).mode,),
        dims=("mode",)
    )
    return coeff

In [10]:
coeff = fitting(ssha_fitting.T, eta_fitting)

ssh_lin = detrend(
    (coeff * eta_mon).sum("mode")
)

### D20 from NEMO 

In [11]:
d20_mon_anom = detrend(
    monthly_anomalies(
        t_grid_file.d20
    )
    .interp(lat=eta.lat,lon = eta.lon)
    .fillna(0.).compute()
)

## Write output

In [12]:
ds = xr.Dataset(
    dict(
        ssh_nemo=ssh_mon_anom,
        d20_nemo=d20_mon_anom,
        ssh_lin=ssh_lin,
    )
)

In [13]:
!rm -rf data_fig2S.zarr
ds.to_zarr("data_fig2S.zarr");